In [2]:
import numpy as np
import pandas as pd 
import os
import matplotlib.pyplot as plt
import tensorflow as tf

import cv2
import random
import time


from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model

from tensorflow.keras.layers import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform

from keras.regularizers import l2
from keras import backend as K
#from tensorflow.keras.engine import Layer, InputSpec

from sklearn.utils import shuffle

from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm


from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from tensorflow.keras.layers import BatchNormalization
from keras.layers import MaxPooling2D
from keras.layers import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.regularizers import l2
from keras import backend as K
#from tensorflow.keras.engine import Layer, InputSpec
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import os

In [3]:
csv_path="E:\datasets\self_food\self_food_data.csv"

In [4]:
train_dir=r"E:\datasets\image_set01\Indian Food Images\Indian Food Images"

In [5]:
food_names=os.listdir(train_dir)

In [6]:
np.array(food_names)

array(['adhirasam', 'aloo_gobi', 'aloo_matar', 'aloo_methi',
       'aloo_shimla_mirch', 'aloo_tikki', 'anarsa', 'ariselu',
       'bandar_laddu', 'basundi', 'bhatura', 'bhindi_masala', 'biryani',
       'boondi', 'butter_chicken', 'chak_hao_kheer', 'cham_cham',
       'chana_masala', 'chapati', 'chhena_kheeri', 'chicken_razala',
       'chicken_tikka', 'chicken_tikka_masala', 'chikki',
       'daal_baati_churma', 'daal_puri', 'dal_makhani', 'dal_tadka',
       'dharwad_pedha', 'doodhpak', 'double_ka_meetha', 'dum_aloo',
       'gajar_ka_halwa', 'gavvalu', 'ghevar', 'gulab_jamun', 'imarti',
       'jalebi', 'kachori', 'kadai_paneer', 'kadhi_pakoda', 'kajjikaya',
       'kakinada_khaja', 'kalakand', 'karela_bharta', 'kofta',
       'kuzhi_paniyaram', 'lassi', 'ledikeni', 'litti_chokha', 'lyangcha',
       'maach_jhol', 'makki_di_roti_sarson_da_saag', 'malapua',
       'misi_roti', 'misti_doi', 'modak', 'mysore_pak', 'naan',
       'navrattan_korma', 'palak_paneer', 'paneer_butter_masala

In [7]:
print(len(food_names))

80


In [8]:
IMG_Shape = (200,200,3)
train_ids = np.arange(0,30)
val_ids = np.arange(30,40)
test_ids = np.arange(40,50)

In [9]:
def loadimgs(names,n = 0,dataset="train"):
    X_train=[]
    X_val = []
    y_train = []
    y_val =[]
    X_test = []
    y_test =[]
    
    img_shape = (200,200)
    cat_dict = {}
    dish_dict = {}
    curr_y = n
    path = train_dir
    
    category_images_tr=[]
    category_images_v=[]
    category_images_te=[] 
        
    clsctr = 0
    for dish in names:
        dish_dict[dish] = clsctr
        dish_path = os.path.join(path,dish) +'/'
        
        #category_images_t=[]
        #category_images_v=[]
                  
        #count = 0 
        #print(len(os.listdir(dish_path)))
        for count, filename in enumerate(os.listdir(dish_path)):
            #print("Class :{}, count: {}".format(clsctr,count)) 
            if count < 30:
                #print("In train condition")
                image_path = os.path.join(dish_path, filename)
                image = cv2.imread(image_path)
                resized_image = cv2.resize(image, img_shape)
                final_image = resized_image #cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
                y_train.append(clsctr)
                try:
                    category_images_tr.append(final_image)
            
                except ValueError as e:
                    print(e)
                    print("error - category_images_train:", final_image)
    
                
            elif count >= 30 and count < 40:
                #print("In val condition")
                image_path = os.path.join(dish_path, filename)
                image = cv2.imread(image_path)
                resized_image = cv2.resize(image, img_shape)
                final_image = resized_image#cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
                y_val.append(clsctr)
                try:
                    category_images_v.append(final_image)
            
                except ValueError as e:
                    print(e)
                    print("error - category_images_test:", final_image)
             
            
            else :
                #print("In test condition")
                image_path = os.path.join(dish_path, filename)
                image = cv2.imread(image_path)
                resized_image = cv2.resize(image, img_shape)
                final_image = resized_image#cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
                y_test.append(clsctr)
                try:
                    category_images_te.append(final_image)
            
                except ValueError as e:
                    print(e)
                    print("error - category_images_test:", final_image)
             
            curr_y += 1    
             
             #count = count + 1
            
        #X_train.append(category_images_t)
        #X_val.append(category_images_v)
        clsctr += 1
    
    #y_train = np.vstack(y_train)
    #y_val = np.vstack(y_val)
    #X_train = np.stack(X_train)
    #X_val = np.stack(X_val)
    X_train = category_images_tr
    X_val = category_images_v
    X_test = category_images_te
    
    #print(y_train)
    
    return X_train,y_train, X_val,y_val, X_test,y_test, dish_dict

In [10]:
X_train, y_train,X_val,y_val,X_test,y_test, dish_dict = loadimgs(food_names)
len(X_train), len(X_val), len(y_train), len(y_val), len(X_test), len(y_test)

(2400, 800, 2400, 800, 800, 800)

In [11]:
def create_pairs(images, labels):
    # initialize two empty lists to hold the (image, image) pairs and
    # labels to indicate if a pair is positive or negative
    random.seed(2021)
    pairImages = []
    pairLabels = []
   
    # calculate the total number of classes present in the dataset
    # and then build a list of indexes for each class label that
    # provides the indexes for all examples with a given label
    numClasses = len(np.unique(y_val))
    classes=np.unique(y_val)
    idx = [np.where(y_val == classes[i]) for i in range(0, numClasses)]
    
    # loop over all images
    for idxA in range(len(images)):
        # grab the current image and label belonging to the current iteration
        currentImage = images[idxA]
        label = labels[idxA]
        
        # randomly pick an image that belongs to the *same* class
        # label
        #posId = random.choice(list(np.where(labels == label)))
        posIdx =random.choice([index for index, element in enumerate(labels) if element == label])
        posImage = images[posIdx]
        
        # prepare a positive pair and update the images and labels
        pairImages.append([currentImage, posImage])
        pairLabels.append([1])
        
        # grab the indices for each of the class labels *not* equal to
        # the current label and randomly pick an image corresponding
        # to a label *not* equal to the current label
        #negId = random.choice(list(np.where(labels != label)))         
        negIdx =random.choice([index for index, element in enumerate(labels) if element != label])
        negImage = images[negIdx]
        
        # prepare a negative pair of images and update our lists
        pairImages.append([currentImage, negImage])
        pairLabels.append([0])
   
    return (np.array(pairImages), np.array(pairLabels))

In [12]:
(pairTrain, labelTrain) = create_pairs(X_train, y_train)
(pairval, labelval) = create_pairs(X_val, y_val)
(pairTest, labelTest) = create_pairs(X_test, y_test)

In [13]:
def tf_siamese_nn(shape, embedding=64, fineTune=False):
    inputs = tf.keras.layers.Input(shape)
    preprocess_fn = preprocess_input
    base_model = tf.keras.applications.vgg19.VGG19(input_shape=shape, include_top=False,                                               weights='imagenet')
    
    if fineTune==False:
        base_model.trainable=False
    else:
        base_model.trainable = True
        # Fine-tune from this layer onwards
        fine_tune_at = len(base_model.layers)-int(len(base_model.layers)*.10)
# Freeze all the layers before the `fine_tune_at` layer
        for layer in base_model.layers[:fine_tune_at]:
          layer.trainable =  False
    x=base_model(inputs)
    x=tf.keras.layers.GlobalAveragePooling2D()(x)
    outputs=tf.keras.layers.Dense(embedding)(x)
    model = tf.keras.Model(inputs, outputs)
    
    return model

model1=tf_siamese_nn(IMG_Shape, 64, True)
model1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 200, 200, 3)]     0         
                                                                 
 vgg19 (Functional)          (None, 6, 6, 512)         20024384  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 64)                32832     
                                                                 
Total params: 20,057,216
Trainable params: 2,392,640
Non-trainable params: 17,664,576
_________________________________________________________________


In [14]:
def euclidean_distance(vectors):
    # unpack the vectors into separate lists
    (featsA, featsB) = vectors
    # compute the sum of squared distances between the vectors
    sumSquared = K.sum(K.square(featsA - featsB), axis=1,keepdims=True)
    # return the euclidean distance between the vectors
    return K.sqrt(K.maximum(sumSquared, K.epsilon()))

In [15]:
img1 = tf.keras.layers.Input(shape=IMG_Shape)
img2 =  tf.keras.layers.Input( shape=IMG_Shape)
featureExtractor = tf_siamese_nn(IMG_Shape)
featsA = featureExtractor(img1)
featsB = featureExtractor(img2)

In [16]:
distance = tf.keras.layers.Lambda(euclidean_distance)([featsA, featsB])

In [17]:
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(distance)
model = tf.keras.Model(inputs=[img1, img2], outputs=outputs)

In [18]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
checkpoint_filepath = r'C:\Users\vaibh\notebooks\khana\checkpoint'
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    save_freq='epoch',
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [20]:
history = model.fit([pairTrain[:, 0], pairTrain[:, 1]], labelTrain[:], validation_data=([pairval[:, 0], pairval[:, 1]], labelval[:]), batch_size=1, epochs=20)

Epoch 1/20
4800/4800 [==============================] - 161s 29ms/step - loss: 7.5611 - accuracy: 0.5658 - val_loss: 0.6049 - val_accuracy: 0.6450
Epoch 2/20
1514/4800 [========>.....................] - ETA: 1:22 - loss: 0.5642 - accuracy: 0.7153

KeyboardInterrupt: 

In [ ]:
preds = model.predict( [pairTest[:,0],pairTest[:,1]])

In [ ]:
preds_ = [int(i > .5) for i in preds]

In [ ]:
labelTest = labelTest.flatten()

In [ ]:
def compute_accuracy(y_true, y_pred):
    correct_predictions = 0
    # iterate over each label and check
    for true, predicted in zip(y_true, y_pred):
        if true == predicted:
            correct_predictions += 1
    # compute the accuracy
    accuracy = correct_predictions/len(y_true)
    return accuracy

In [ ]:
compute_accuracy(labelTest,preds_)

In [ ]:
def loadimgs_(names):
    '''
    path => Path of train directory or test directory
    '''
    images = []
    labels = [] 
    
    img_shape = (200,200)
    dish_dict = {}
    
    path = train_dir
    
    category_images = []
        
    clsctr = 0
    for dish in names:
        dish_dict[dish] = clsctr
        dish_path = os.path.join(path,dish) +'/'
        
        for count, filename in enumerate(os.listdir(dish_path)):
            #print("Class :{}, count: {}".format(clsctr,count)) 
            if count == 1:
                #print("In train condition")
                image_path = os.path.join(dish_path, filename)
                image = cv2.imread(image_path)
                resized_image = cv2.resize(image, img_shape)
                final_image = resized_image #cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
                labels.append(clsctr)
                try:
                    category_images.append(final_image)
            
                except ValueError as e:
                    print(e)
                    print("error - category_images_train:", final_image)
            else:
                pass

        clsctr += 1
        
    images = category_images
    
    return images, labels, dish_dict

In [ ]:
images_, labels_ , dish_dict_ = loadimgs_(food_names) 

In [ ]:
def create_pairs_for_pred(image_path, images_):
    # initialize two empty lists to hold the (image, image) pairs and
    # labels to indicate if a pair is positive or negative
    random.seed(2021)
    pairImages = []
    pairLabels = []
   
    image = cv2.imread(image_path)
    resized_image = cv2.resize(image, (200,200))
                
    # loop over all images
    for idxA in range(len(images_)):
    
        currentImage = images_[idxA]

        pairImages.append([currentImage, resized_image]) 
   
    return (np.array(pairImages))

In [ ]:
image_path = r"D:\New folder\download (1).jpg"
pairPred = create_pairs_for_pred(image_path, images_)

In [ ]:
image_prediction = model.predict( [pairPred[:,0],pairPred[:,1]])

In [ ]:
final_food_name=list(dish_dict.keys())[list(dish_dict.values()).index(np.argmax(image_prediction))]

In [ ]:
print("The image is of {}".format(list(dish_dict.keys())[list(dish_dict.values()).index(np.argmax(image_prediction))])) 

In [ ]:
plt.imshow()